# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.



In [0]:
!pip install syft
!pip install numpy

     |████████████████████████████████| 215kB 4.8MB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
     |████████████████████████████████| 1.4MB 46.1MB/s 
     |████████████████████████████████| 256kB 47.9MB/s 
     |████████████████████████████████| 389kB 46.1MB/s 
     |████████████████████████████████| 71kB 28.7MB/s 
     |████████████████████████████████| 450kB 40.2MB/s 
     |████████████████████████████████| 276kB 46.7MB/s 
     |████████████████████████████████| 51kB 11.0MB/s 
     |████████████████████████████████| 122kB 47.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/ad/9a/f4/3105b5209674ac77fcca7fede95184c62a95df0196888e0e76
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built zstd pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: msgpack 0.5.6
    Uninstalling msgpack-0.5.6:
     

In [3]:
import torch as th

In [33]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [34]:
y = x + x

In [35]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [4]:
import syft as sy

In [5]:
hook = sy.TorchHook(th)  # this line of code create an instance of th with its backend APIs modified with Syft functions

In [38]:
th.tensor([1,2,3,4,5]) # notice that Torch functionalities still behave the same

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [8]:
bob = sy.VirtualWorker(hook, id="bob") # creates a virtual worker (a simulaion to interface to Bob machines)

In [60]:
bob._objects

{}

In [61]:
x = th.tensor([1,2,3,4,5])

In [62]:
x_ptr = x.send(bob) # send this data to bob

In [63]:
bob._objects

{96938502639: tensor([1, 2, 3, 4, 5])}

In [64]:
x

tensor([1, 2, 3, 4, 5])

In [66]:
x_ptr.location

<VirtualWorker id:bob #objects:1>

In [67]:
x_ptr.id_at_location

96938502639

In [68]:
x.id

96938502639

In [69]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [70]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [71]:
x = x_ptr.get()
x

tensor([1, 2, 3, 4, 5])

In [72]:
bob._objects

{}

# Project: Experience with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [21]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id="alice")

In [22]:
# 1 - create some data (a tensor)
y = th.tensor([6, 7, 8, 9, 10])
# 2- send the data to bob and alice
y_ptr = y.send(bob, alice)


In [23]:
# notice what does the send fucntion on two workers return; a multi-pointer
y_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:15720494150 -> bob:66987003815]
	-> (Wrapper)>[PointerTensor | me:38759189566 -> alice:24699885404]

In [24]:
# what does .child on the pointer object return?
y_ptr_child = y_ptr.child()

# what does .child.child on the pointer object return? 

TypeError: 'MultiPointerTensor' object is not callable

In [0]:
# try the .get() on the pointer


In [0]:
# try the .get(sum_results=True) on your pointer


In [32]:
# create a second Virtual worker and call it Alice
alice = sy.VirtualWorker(hook, id="alice")

In [74]:
# 1 - create some data (a tensor)
y = th.tensor([6, 7, 8, 9, 10])

In [75]:
# 2- send the data to bob and alice
y_ptr = y.send(bob, alice)

In [76]:
# notice what does the send fucntion on two workers return; a multi-pointer
y_ptr

(Wrapper)>[MultiPointerTensor]
	-> (Wrapper)>[PointerTensor | me:3895179973 -> bob:44169648647]
	-> (Wrapper)>[PointerTensor | me:70568425793 -> alice:17544941549]

In [79]:
# what does .child on the pointer object return?
y_ptr.child

MultiPointerTensor>{'bob': (Wrapper)>[PointerTensor | me:3895179973 -> bob:44169648647], 'alice': (Wrapper)>[PointerTensor | me:70568425793 -> alice:17544941549]}

In [78]:
# what does .child.child on the pointer object return?
y_ptr.child.child

{'bob': (Wrapper)>[PointerTensor | me:3895179973 -> bob:44169648647],
 'alice': (Wrapper)>[PointerTensor | me:70568425793 -> alice:17544941549]}

In [80]:
# try the .get() on the pointer
y = y_ptr.get()
y

[tensor([ 6,  7,  8,  9, 10]), tensor([ 6,  7,  8,  9, 10])]

In [83]:
# try the .get(sum_results=True) on your pointer
y = th.tensor([6, 7, 8, 9, 10])
y_ptr = y.send(bob, alice)
y_ptr.get(sum_results=True)


tensor([12, 14, 16, 18, 20])

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [0]:
x # a pointer to a remote tensor located at bob

(Wrapper)>[PointerTensor | me:96502616908 -> bob:83999397113]

In [0]:
y # a pointer to another remote tensor located at bob

(Wrapper)>[PointerTensor | me:35900922850 -> bob:59092289169]

In [0]:
z = x + y # treat those tensors (i.e., x and y) as local tensors, but they are actually executed remotely
z

(Wrapper)>[PointerTensor | me:12996980457 -> bob:54222168809]

In [0]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
z = th.add(x,y) # we can also use Torch operation on these tensors to be executed remotely
z

(Wrapper)>[PointerTensor | me:28437210120 -> bob:28437210120]

In [0]:
z = z.get() # yup! same results!
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()
z = z.get()
z

tensor(20., requires_grad=True)

In [0]:
z.backward() 

tensor(20., requires_grad=True)

In [0]:
x = x.get()

In [0]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [0]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). 

Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [20]:
from torch import nn

# create some toy data for our model
input_data = th.tensor([[1.,1],[0.5,1],[1,0],[0,0]], requires_grad=True)
output_data = th.tensor([[1.],[1],[0],[0]], requires_grad=True)


In [21]:
# send the data to bob
input_ptr = input_data.send(bob)
labels_ptr = output_data.send(bob)

In [22]:
# create some linear weights and send them to bob
epochs = 2
learning_rate = 0.001

weights1 = th.tensor([[.5], [1]], requires_grad=True)
##weights2 = th.tensor()



w1_ptr = weights1.send(bob)
##w2_ptr = weights2.send(bob)


In [23]:
# create a linear model and train it on Bob's machine
# remember how to create a linear model? No? :( 
# Here's the main steps:
#    1- find a prediction
#    2- calcualte the loss (a mean square loss)
#    3- backpropogate using the backword() function
#    4- DO NOT forget to clear your gradients after updating the weights
#    weights.data.sum_(weights.grad * lr)

criterion = nn.MSELoss()
##crit_ptr = criterion.send(bob)

output_ptr = th.mm(input_ptr, w1_ptr)
## output = output.get()
output_ptr

(Wrapper)>[PointerTensor | me:40892866015 -> bob:75320090685]

In [24]:
## calculate the loss
loss_ptr = criterion(output_ptr, labels_ptr)
##loss = loss.get()
loss_ptr.backward()


weights = w1_ptr.get()
print(weights)

weights.grad = th.zeros(2,1).float()

weights = weights.data + (weights.grad * learning_rate)
print(weights)

tensor([[0.5000],
        [1.0000]], requires_grad=True)
tensor([[0.5000],
        [1.0000]])


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{23831414651: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects

{92212512027: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [0]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [0]:
bob._objects  # some error from Jupyter is casuing this to appear!

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [0]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [0]:
bob = bob.clear_objects() # erase force 
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
bob._objects # notice that there is only a single tensor in bob

{17426510898: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [0]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [25]:
from torch import nn, optim

In [26]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [27]:
# A Toy Model
model = nn.Linear(2,1)

In [28]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [29]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.7816)
tensor(0.4550)
tensor(0.2497)
tensor(0.1645)
tensor(0.1116)
tensor(0.0764)
tensor(0.0528)
tensor(0.0367)
tensor(0.0258)
tensor(0.0182)
tensor(0.0130)
tensor(0.0093)
tensor(0.0067)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0020)
tensor(0.0015)
tensor(0.0011)
tensor(0.0008)


In [30]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [33]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [34]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [35]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [36]:
train()

tensor(0.1682, requires_grad=True)
tensor(0.4907, requires_grad=True)
tensor(0.2001, requires_grad=True)
tensor(0.3438, requires_grad=True)
tensor(0.1539, requires_grad=True)
tensor(0.2493, requires_grad=True)
tensor(0.1150, requires_grad=True)
tensor(0.1819, requires_grad=True)
tensor(0.0860, requires_grad=True)
tensor(0.1333, requires_grad=True)
tensor(0.0643, requires_grad=True)
tensor(0.0980, requires_grad=True)
tensor(0.0482, requires_grad=True)
tensor(0.0722, requires_grad=True)
tensor(0.0361, requires_grad=True)
tensor(0.0534, requires_grad=True)
tensor(0.0271, requires_grad=True)
tensor(0.0395, requires_grad=True)
tensor(0.0203, requires_grad=True)
tensor(0.0293, requires_grad=True)
tensor(0.0153, requires_grad=True)
tensor(0.0218, requires_grad=True)
tensor(0.0115, requires_grad=True)
tensor(0.0162, requires_grad=True)
tensor(0.0086, requires_grad=True)
tensor(0.0120, requires_grad=True)
tensor(0.0065, requires_grad=True)
tensor(0.0090, requires_grad=True)
tensor(0.0048, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [37]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

<VirtualWorker id:alice #tensors:0>

In [38]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [39]:
x = x.send(alice)

In [40]:
bob._objects

{48367049983: tensor([1, 2, 3, 4, 5])}

In [41]:
alice._objects

{95980182809: (Wrapper)>[PointerTensor | alice:95980182809 -> bob:48367049983]}

In [42]:
y = x + x

In [43]:
y

(Wrapper)>[PointerTensor | me:33723436075 -> alice:8788668325]

In [44]:
bob._objects

{48367049983: tensor([1, 2, 3, 4, 5]),
 24834062711: tensor([ 2,  4,  6,  8, 10])}

In [45]:
alice._objects

{95980182809: (Wrapper)>[PointerTensor | alice:95980182809 -> bob:48367049983],
 8788668325: (Wrapper)>[PointerTensor | alice:8788668325 -> bob:24834062711]}

In [46]:
jon = sy.VirtualWorker(hook, id="jon")

In [47]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [48]:
bob._objects

{7217444142: tensor([1, 2, 3, 4, 5])}

In [49]:
alice._objects

{99141699014: (Wrapper)>[PointerTensor | alice:99141699014 -> bob:7217444142]}

In [50]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:99141699014 -> bob:7217444142]

In [51]:
bob._objects

{7217444142: tensor([1, 2, 3, 4, 5])}

In [52]:
alice._objects

{}

In [53]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [54]:
bob._objects

{}

In [55]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [56]:
bob._objects

{21849805925: tensor([1, 2, 3, 4, 5])}

In [57]:
alice._objects

{15452772752: (Wrapper)>[PointerTensor | alice:15452772752 -> bob:21849805925]}

In [58]:
del x

In [59]:
bob._objects

{}

In [60]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [61]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [62]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{27218003307: tensor([1, 2, 3, 4, 5])}

In [64]:
alice._objects

{}

In [65]:
x.move(alice)

(Wrapper)>[PointerTensor | me:59299972562 -> alice:59299972562]

In [66]:
bob._objects

{}

In [67]:
alice._objects

{59299972562: tensor([1, 2, 3, 4, 5])}

In [68]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [69]:
bob._objects

{8617479081: tensor([1, 2, 3, 4, 5])}

In [70]:
alice._objects

{59299972562: tensor([1, 2, 3, 4, 5]),
 83315228467: (Wrapper)>[PointerTensor | alice:83315228467 -> bob:8617479081]}

In [71]:
x.remote_get()

(Wrapper)>[PointerTensor | me:62013719924 -> alice:83315228467]

In [72]:
bob._objects

{}

In [74]:
alice._objects

{59299972562: tensor([1, 2, 3, 4, 5]), 83315228467: tensor([1, 2, 3, 4, 5])}

In [75]:
x.move(bob)

(Wrapper)>[PointerTensor | me:62013719924 -> bob:62013719924]

In [76]:
x

(Wrapper)>[PointerTensor | me:62013719924 -> bob:62013719924]

In [77]:
bob._objects

{62013719924: tensor([1, 2, 3, 4, 5])}

In [78]:
alice._objects

{59299972562: tensor([1, 2, 3, 4, 5])}